In [1]:
import pandas as pd
import numpy as np
from disc_dataset_equalwidth import disc_dataset_equalwidth
from Group_JMI_Rand import Group_JMI_Rand
from Estabilidad import nogueria_test

In [9]:
#import data
X_org=pd.read_feather(path="/Users/macondo/Documents/GitHub/TFM/Raw_Data/pathvals_gtex-v8_edger-v3-40-0_hipathia-norm-v2-14-0.feather")
labels_org=pd.read_feather(path="/Users/macondo/Documents/GitHub/TFM/Raw_Data/labels.feater")

X=X_org.to_numpy()
Y_labels=labels_org.to_numpy()

X=X[0:100,1:40]
Y_labels=Y_labels[0:100,1:26]

In [10]:
X

array([[0.15977083600231282, 0.08501765008863131, 0.07174244549734941,
        ..., 0.15852774217475615, 0.2113327886866544,
        0.17862195143948495],
       [0.09682922661900883, 0.1302983257032947, 0.09235770431204611,
        ..., 0.11862212021367934, 0.31403439128478267,
        0.22452905684816288],
       [0.07639215205010023, 0.09383911834339116, 0.09145582464919537,
        ..., 0.11746903187528945, 0.24652923664418552,
        0.2655078753980535],
       ...,
       [0.225885255534676, 0.054898401074099094, 0.025942772704988848,
        ..., 0.14947006924569342, 0.1554174510008308,
        0.19144283410749263],
       [0.3891475705101022, 0.45484191681177005, 0.02605400094868829,
        ..., 0.21922245653233333, 0.128930065519743, 0.0474715555844687],
       [0.056314365445542074, 0.06036541157683491, 0.009416107965851902,
        ..., 0.10432294152870229, 0.17953638968204227,
        0.1186470433607706]], dtype=object)

In [11]:
Y_labels

array([[0.00376202, 0.00182869, 0.0026343 , ..., 0.08788025, 0.14052651,
        0.02516181],
       [0.00280934, 0.00107259, 0.00136752, ..., 0.1006948 , 0.10683821,
        0.01773169],
       [0.00429877, 0.00233888, 0.00205188, ..., 0.08392364, 0.13182199,
        0.02582674],
       ...,
       [0.00282596, 0.00150849, 0.00191896, ..., 0.09432988, 0.14694872,
        0.01607587],
       [0.00322053, 0.00175059, 0.00187832, ..., 0.10403361, 0.10821894,
        0.01474857],
       [0.00278516, 0.00110509, 0.00213564, ..., 0.10341365, 0.137018  ,
        0.02316267]])

In [12]:
bins=5
X_inputs_disc = disc_dataset_equalwidth(X, bins)
X_inputs_disc

array([[2., 0., 1., ..., 2., 1., 2.],
       [1., 1., 1., ..., 1., 3., 3.],
       [0., 0., 1., ..., 1., 2., 4.],
       ...,
       [2., 0., 0., ..., 1., 1., 3.],
       [4., 4., 0., ..., 3., 0., 0.],
       [0., 0., 0., ..., 0., 1., 1.]])

In [27]:
#Con la distancia euclidiana
topK=10
np.random.seed(1026) 
Selected_with_Group_JMI_1 = Group_JMI_Rand(X_inputs_disc,Y_labels, topK, 'euclidean')
Selected_with_Group_JMI_1

[37, 8, 38, 27, 23, 14, 15, 26, 20, 28]

In [31]:
#Con la distancia de hamming
topK=10
np.random.seed(1026) 
Selected_with_Group_JMI_2 = Group_JMI_Rand(X_inputs_disc,Y_labels, topK, 'hamming')
Selected_with_Group_JMI_2

[0, 38, 4, 16, 36, 6, 15, 24, 14, 27]

In [39]:
#Funcion para probar la estabilidad en "n_samples" muestras de tamanho 50 cada una   
def estabilidad(n_samples, X, Y_labels, bins, topK):
    np.random.seed(1026)
    estab_matriz = np.zeros((n_samples, X.shape[1]))
    for sample in range(n_samples):
        random1=np.random.choice(X.shape[0], 50, replace=False)
        X_rd=X[random1,:]
        Y_labels_rd=Y_labels[random1,:]
        X_inputs_disc = disc_dataset_equalwidth(X_rd, bins)
        Selected_with_Group_JMI_1 = Group_JMI_Rand(X_inputs_disc,Y_labels_rd, topK, 'euclidean')
        estab_matriz[sample, Selected_with_Group_JMI_1] = 1
    return estab_matriz

In [42]:
#Prueba de la estabilidad en 5 muestras
#Bins=5
#topK=10
estab_matriz=estabilidad(5, X, Y_labels, 5, 10)
estab_matriz

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0.,
        0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0.,
        1., 0., 1., 1., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
        1., 1., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0.,
        1., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        1., 0., 0., 0., 1., 1., 1.]])

In [43]:
#Nogueria score
nogueria_score=nogueria_test(estab_matriz)
nogueria_score

NogueiraTest(estimator=0.36793103448275855, upper=0.4729410028253467, lower=0.2629210661401704, var=0.0028705483947681193, error=0.10500996834258813, alpha=0.05)